<h1 style="text-align: center;">Formula 1 Race predictor</h1>

<h3 style="text-align: center; font-weight: normal; color: gray;">Petar Stamenkovic &nbsp; | &nbsp; Aleksa Mitrovcan</h3>

<img src="Image/lec.jpeg" style="display: block; margin-left: auto; margin-right: auto; width: 100%;" />



## Intro

Hello everyone! This is our second project together, and we decided to try something different when it comes to Formula 1. In this notebook, we will use Gradient Boosting (specifically XGBoost) to try and predict the final race position for each driver on the grid. The user will input a track for prediction and fetch data for each year from 2022 to 2024 if the race happened in any of those years. We feel like the 3-year gap is solid, but this could be easily modified if you wish, simply by calling a certain function a couple more times.

For this project, just like with the previous one, we used Python and Jupyter Notebook for better user readability of the final product. Image below shows the basic way XGBoost works.

<img src="Image/xgb.png" style="display: block; margin-left: auto; margin-right: auto; width: 100%;" />


## Importing packages

Firstly, we want to import all necessary packages. We will use Fast F1 for Python and sklearn package that contains some elementary functions for machine learning model creation. 

After importing, we enable caching, so that data fetching process is faster. Then we import our Driver data table, that contains each driver experience in number of race starts, and we set a track name for prediction. 

In [1]:
# Importing all packages and loading neccesary data
import os
import fastf1
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
import shutil

# Enable cache memory for faster access
custom_cache_folder = "f1_cache"
os.makedirs(custom_cache_folder, exist_ok=True)
fastf1.Cache.enable_cache(custom_cache_folder)

# Import driver info table containing years of experience for each driver
experience = pd.read_csv("Dataset/DriverInfo.csv")

# Set a race name for prediction 
circuit_name = "Japanese Grand Prix"

## Necessary functions

For the better scalability and modularity, we created functions for fetching our data. Please be aware that fast f1 will **autocorrect** your input in case the race did not happen in one of the years. For example, Chinese Grand Prix was not held in 2022, so fast f1 will change it to Japanese Grand Prix, for that year only. Obviously, we do not want this to happen, as that means we are corrupting our training set. Unfortunately, this does mean that for some races, MAE score will be worse and result in less accurate predictions, but as mentioned, this can be possibly improved by adding more years into the mix.

In [2]:
# This function ensures that right races are loaded in training set
# In case that selected race did not happen in one of 3 years we skip it
def race_exists(year, circuit):
    try:
        schedule = fastf1.get_event_schedule(year)
        circuit_lower = circuit.strip().lower()
        event_names = schedule['EventName'].str.strip().str.lower()
        return circuit_lower in event_names.values
    except Exception as e:
        print(f"Error checking race schedule for {year}: {e}")
        return False

# This function fetches selected race data and creates a data frame        
def get_race_data(year, circuit):
    if not race_exists(year, circuit):
        print(f"Race '{circuit}' not found in {year}. Skipping.")
        return pd.DataFrame()

    try:
        # Fast F1 API main call
        session = fastf1.get_session(year, circuit, 'R')
        session.load()

        # Loading all driver lap values
        all_drivers = session.laps["Driver"].unique()
        fastest_laps = []
        
        # Fetching fastest laps for all drivers
        for driver in all_drivers:
            try:
                driver_laps = session.laps.pick_drivers(driver)
                if not driver_laps.empty:
                    fastest_lap = driver_laps.pick_fastest()
                    if not pd.isna(fastest_lap['LapTime']):
                        fastest_lap_seconds = fastest_lap['LapTime'].total_seconds()
                        fastest_laps.append([driver, fastest_lap_seconds])
                    else:
                        fastest_laps.append([driver, None])
                else:
                    fastest_laps.append([driver, None])
            except Exception as e:
                print(f"Error processing driver {driver} in {year}: {e}")
                fastest_laps.append([driver, None])

        fastest_lap_df = pd.DataFrame(fastest_laps, columns=["Abbreviation", "FastestLap"])

        # Extract only necessary information from inital table
        results = session.results[["FullName", "Abbreviation", "TeamName", "GridPosition", "Position", "Status"]]
        results["Year"] = year
        
        # Loading weather information and calculating average values
        weather_data = session.weather_data
        results["Humidity"] = weather_data["Humidity"].mean()
        results["Temperature"] = weather_data["AirTemp"].mean()
        results["Rain"] = weather_data["Rainfall"].mean()

        results = results.merge(fastest_lap_df, on="Abbreviation")
        return results
    
    except Exception as e:
        print(f"Error getting race data for {year} {circuit}: {e}")
        return pd.DataFrame()

        
# This function fetches qualifing data
def get_qualifying_data(year, circuit):
    
    if not race_exists(year, circuit):
        print(f"Race '{circuit}' not found in {year}. Skipping.")
        return pd.DataFrame()
    
    try:
        session = fastf1.get_session(year, circuit, 'Q')
        session.load()

        all_drivers = session.laps["Driver"].unique()
        qualifying_data = []

        for driver in all_drivers:
            try:
                driver_laps = session.laps.pick_drivers(driver)
                if not driver_laps.empty:
                    fastest_lap = driver_laps.pick_fastest()
                    qualifying_time = fastest_lap['LapTime'].total_seconds() if 'LapTime' in fastest_lap else None
                    speed_trap = fastest_lap['SpeedI2'] if 'SpeedI2' in fastest_lap else None
                else:
                    qualifying_time = None
                    speed_trap = None
                
                qualifying_data.append([driver, qualifying_time, speed_trap])
            except Exception as e:
                print(f"Error processing driver {driver} in {year}: {e}")
                qualifying_data.append([driver, None, None])

        qualifying_df = pd.DataFrame(
            qualifying_data,
            columns=["Abbreviation", "QualifyingTime", "SpeedTrapQualy"]
        )
        
        results = session.results[["Abbreviation", "Position"]]
        results = results.rename(columns={"Position": "QualifyingPosition"})
        qualifying_results = results.merge(qualifying_df, on="Abbreviation")
        qualifying_results["Year"] = year
        
        return qualifying_results
    
    except Exception as e:
        print(f"Error getting qualifying data for {year} {circuit}: {e}")
        return pd.DataFrame()

## Function calls 

This is where we call our functions for data fetching and data frame creation. Here is the chance to increase the number of years for your model training, if necessary.

In [3]:
# Get data for all years for both races and qualifying
race_2022 = get_race_data(2022, circuit_name)
race_2023 = get_race_data(2023, circuit_name)
race_2024 = get_race_data(2024, circuit_name)

quali_2022 = get_qualifying_data(2022, circuit_name)
quali_2023 = get_qualifying_data(2023, circuit_name)
quali_2024 = get_qualifying_data(2024, circuit_name)

core           INFO 	Loading data for Japanese Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '31', '44', '5', '14', '63', '6', '4', '3', '18', '22', '20', '77', '24', '47', '10', '55', '23']
/tmp/ipykernel_19843/3593158436.py:49: SettingWithCopyWarning:

Error processing driver ALB in 2022: 'NoneType' object is not subscriptable
Error processing driver SAI in 2022: 'NoneType' object is not subscriptable


core           INFO 	Loading data for Japanese Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 1 completed the race distance 00:00.076000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '81', '16', '44', '55', '63', '14', '31', '10', '40', '2

Error processing driver RIC in 2024: 'NoneType' object is not subscriptable
Error processing driver ALB in 2024: 'NoneType' object is not subscriptable


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '55', '11', '31', '44', '14', '63', '5', '4', '3', '77', '22', '24', '47', '23', '10', '20', '18', '6']
core           INFO 	Loading data for Japanese Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data


Error processing driver SAR in 2023: argument of type 'NoneType' is not iterable


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '4', '55', '14', '81', '44', '16', '63', '22', '3', '27', '77', '23', '31', '18', '10', '20', '2', '24']


## Format and merge necessary data

After fetching the data, we merge them all in one big final Data Frame. Merge is mostly done on driver Abbreviations. For features Time Ratio and Speed Trap, we group our entries by drivers and calculate the mean values for each one. This is later used in the prediction sample.

In [4]:
# Combine all data
all_races = pd.concat([race_2022, race_2023, race_2024], ignore_index=True)
all_qualis = pd.concat([quali_2022, quali_2023, quali_2024], ignore_index=True)

# Merge race and qualifying data
merged_data = pd.merge(
    all_races,
    all_qualis,
    on=["Abbreviation", "Year"],
    how="left"
)

# Add driver experience information
final_data = merged_data.merge(experience, on=["Abbreviation"])

# Calculate TimeRatio (QualifyingTime / FastestLap)
final_data["TimeRatio"] = final_data.apply(
    lambda row: row["QualifyingTime"] / row["FastestLap"] 
    if pd.notna(row["QualifyingTime"]) and pd.notna(row["FastestLap"]) and row["FastestLap"] != 0 
    else None,
    axis=1
)

# Clean up the final DataFrame and reorganize it
final_data = final_data[[
    'Year', 'FullName', 'Abbreviation', 'TeamName', 
    'QualifyingPosition', 'GridPosition', 'Position',
    'QualifyingTime', 'FastestLap', 'TimeRatio', 'SpeedTrapQualy',
    'Temperature', 'Humidity', 'Rain', 'Status','NumberOfRaces'
    # Add any columns from experience.csv that you need
]]

# This mapping is neccesary since these teams changed names during these years
team_mappings = {
    'Alfa Romeo': 'Kick Sauber',
    'AlphaTauri': 'RB'
}

final_data['TeamName'] = final_data['TeamName'].replace(team_mappings)
final_data

,Year,FullName,Abbreviation,TeamName,QualifyingPosition,GridPosition,Position,QualifyingTime,FastestLap,TimeRatio,SpeedTrapQualy,Temperature,Humidity,Rain,Status,NumberOfRaces
0,2022,Max Verstappen,VER,Red Bull Racing,1.0,1.0,1.0,89.304,104.911,0.851236,304.0,16.717647,86.211765,0.980392,Finished,211
1,2022,Sergio Perez,PER,Red Bull Racing,4.0,4.0,2.0,89.709,106.120,0.845354,303.0,16.717647,86.211765,0.980392,Finished,281
2,2022,Charles Leclerc,LEC,Ferrari,2.0,2.0,3.0,89.314,104.489,0.854769,300.0,16.717647,86.211765,0.980392,Finished,149
3,2022,Esteban Ocon,OCO,Alpine,5.0,5.0,4.0,90.165,106.559,0.846151,300.0,16.717647,86.211765,0.980392,Finished,158
4,2022,Lewis Hamilton,HAM,Mercedes,6.0,6.0,5.0,90.261,105.530,0.855311,294.0,16.717647,86.211765,0.980392,Finished,358
5,2022,Sebastian Vettel,VET,Aston Martin,9.0,9.0,6.0,90.554,106.964,0.846584,295.0,16.717647,86.211765,0.980392,Finished,300
6,2022,Fernando Alonso,ALO,Alpine,7.0,7.0,7.0,90.322,104.412,0.865054,300.0,16.717647,86.211765,0.980392,Finished,405
7,2022,George Russell,RUS,Mercedes,8.0,8.0,8.0,90.389,107.004,0.844725,295.0,16.717647,86.211765,0.980392,Finished,130
8,2022,Nicholas Latifi,LAT,Williams,20.0,20.0,9.0,91.511,108.371,0.844423,308.0,16.717647,86.211765,0.980392,Finished,61
9,2022,Lando Norris,NOR,McLaren,10.0,10.0,10.0,90.473,108.175,0.836358,298.0,16.717647,86.211765,0.980392,Finished,130


## Encoding string type features

Machine learning models do not work with string values, so here we map the drivers manually, since they also changed during our year gap. Teams did not change, so we are free to use Label encoder to do the mapping.

In [5]:
from sklearn.preprocessing import LabelEncoder

# Create a copy so we don't modify the original
final_sessions_encoded = final_data.copy()

# Encoding team names using Label Encoder
label_encoder_team = LabelEncoder()
final_sessions_encoded["TeamName"] = label_encoder_team.fit_transform(final_sessions_encoded["TeamName"])

# Driver mapping
driver_encoding = {
    'ALB': 0,   # Alexander Albon
    'ALO': 1,   # Fernando Alonso
    'BOT': 2,   # Valtteri Bottas
    'COL': 3,   # (Reserved/Historical)
    'DOO': 4,   # (Reserved/Historical)
    'GAS': 5,   # Pierre Gasly
    'HAM': 6,   # Lewis Hamilton
    'HUL': 7,   # Nico Hülkenberg
    'LAT': 8,   # Nicholas Latifi (2022)
    'LAW': 9,   # Liam Lawson (2023 substitute)
    'LEC': 10,  # Charles Leclerc
    'MAG': 11,  # Kevin Magnussen
    'NOR': 12,  # Lando Norris
    'OCO': 13,  # Esteban Ocon
    'PER': 14,  # Sergio Pérez
    'PIA': 15,  # Oscar Piastri
    'RIC': 16,  # Daniel Ricciardo
    'RUS': 17,  # George Russell
    'SAI': 18,  # Carlos Sainz
    'SAR': 19,  # (Reserved/Historical)
    'STR': 20,  # Lance Stroll
    'TSU': 21,  # Yuki Tsunoda
    'VER': 22,  # Max Verstappen
    'VET': 23,  # Sebastian Vettel (retired after 2022)
    'ZHO': 24,  # Zhou Guanyu
    'BEA': 25,  # Oliver Bearman
    'ANT': 26,  # Kimi Antonelli
    'BOR': 27,  # Gabriel Bortoleto
    'HAD': 28   # Isaac Hadjar
}

# Mapping for Teams
team_mapping = dict(zip(label_encoder_team.classes_, label_encoder_team.transform(label_encoder_team.classes_)))
print("Team Encoding Mapping:", team_mapping)

final_sessions_encoded['Abbreviation'] = final_sessions_encoded['Abbreviation'].replace(driver_encoding)

Team Encoding Mapping: {'Alpine': np.int64(0), 'Aston Martin': np.int64(1), 'Ferrari': np.int64(2), 'Haas F1 Team': np.int64(3), 'Kick Sauber': np.int64(4), 'McLaren': np.int64(5), 'Mercedes': np.int64(6), 'RB': np.int64(7), 'Red Bull Racing': np.int64(8), 'Williams': np.int64(9)}


/tmp/ipykernel_19843/1199710730.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_sessions_encoded['Abbreviation'] = final_sessions_encoded['Abbreviation'].replace(driver_encoding)


## Resolve possible NaN values

In a table above, we can see that there are couples of NaN values. Since NaN value are possible to occur here for various reasons like driver not finishing the race or not attending qualification session, we resolve this by calculating the mean value for the problematic column and fill those NaN values. This seems like the valid solution.

In [6]:
# Check for NaN values
print(final_sessions_encoded.isnull().sum())

# Fill NaN values with mean values in those columns
for col in ["QualifyingTime", "FastestLap", "TimeRatio", "SpeedTrapQualy"]:
    final_sessions_encoded[col] = final_sessions_encoded[col].fillna(final_sessions_encoded[col].mean())

# Verify that there are no NaN values
print(final_sessions_encoded.isnull().sum())

# Calculate and print mean trap speed for each driver - this is visualy easier for creating prediction sample
mean_speeds = final_sessions_encoded.groupby('FullName')['SpeedTrapQualy'].mean().reset_index()
print("Mean speeds are: ", mean_speeds)

# Calculate and print time ratio for each driver - this is visualy easier for creating prediction sample
mean_time_ratio = final_sessions_encoded.groupby('FullName')['TimeRatio'].mean().reset_index()
print("Mean time ratio's are: ", mean_time_ratio)

# Calculating mean values and importing them into Data Frame
final_sessions_encoded['MeanSpeedTrap'] = final_sessions_encoded.groupby('FullName')['SpeedTrapQualy'].transform('mean')
final_sessions_encoded['MeanTimeRatio'] = final_sessions_encoded.groupby('FullName')['TimeRatio'].transform('mean')
final_sessions_encoded['YearWeight'] = final_sessions_encoded['Year'].map({2024: 4, 2023: 2, 2022: 1})

final_sessions_encoded

Year                  0
FullName              0
Abbreviation          0
TeamName              0
QualifyingPosition    0
GridPosition          0
Position              0
QualifyingTime        1
FastestLap            4
TimeRatio             5
SpeedTrapQualy        1
Temperature           0
Humidity              0
Rain                  0
Status                0
NumberOfRaces         0
dtype: int64
Year                  0
FullName              0
Abbreviation          0
TeamName              0
QualifyingPosition    0
GridPosition          0
Position              0
QualifyingTime        0
FastestLap            0
TimeRatio             0
SpeedTrapQualy        0
Temperature           0
Humidity              0
Rain                  0
Status                0
NumberOfRaces         0
dtype: int64
Mean speeds are:              FullName  SpeedTrapQualy
0    Alexander Albon      306.000000
1       Carlos Sainz      300.666667
2    Charles Leclerc      299.333333
3   Daniel Ricciardo      297.500000
4  

,Year,FullName,Abbreviation,TeamName,QualifyingPosition,GridPosition,Position,QualifyingTime,FastestLap,TimeRatio,SpeedTrapQualy,Temperature,Humidity,Rain,Status,NumberOfRaces,MeanSpeedTrap,MeanTimeRatio,YearWeight
0,2022,Max Verstappen,22,8,1.0,1.0,1.0,89.304000,104.911000,0.851236,304.000000,16.717647,86.211765,0.980392,Finished,211,302.000000,0.912036,1
1,2022,Sergio Perez,14,8,4.0,4.0,2.0,89.709000,106.120000,0.845354,303.000000,16.717647,86.211765,0.980392,Finished,281,302.333333,0.894678,1
2,2022,Charles Leclerc,10,2,2.0,2.0,3.0,89.314000,104.489000,0.854769,300.000000,16.717647,86.211765,0.980392,Finished,149,299.333333,0.906050,1
3,2022,Esteban Ocon,13,0,5.0,5.0,4.0,90.165000,106.559000,0.846151,300.000000,16.717647,86.211765,0.980392,Finished,158,301.666667,0.903162,1
4,2022,Lewis Hamilton,6,6,6.0,6.0,5.0,90.261000,105.530000,0.855311,294.000000,16.717647,86.211765,0.980392,Finished,358,298.000000,0.913488,1
5,2022,Sebastian Vettel,23,1,9.0,9.0,6.0,90.554000,106.964000,0.846584,295.000000,16.717647,86.211765,0.980392,Finished,300,295.000000,0.846584,1
6,2022,Fernando Alonso,1,0,7.0,7.0,7.0,90.322000,104.412000,0.865054,300.000000,16.717647,86.211765,0.980392,Finished,405,301.000000,0.910657,1
7,2022,George Russell,17,6,8.0,8.0,8.0,90.389000,107.004000,0.844725,295.000000,16.717647,86.211765,0.980392,Finished,130,299.000000,0.903813,1
8,2022,Nicholas Latifi,8,9,20.0,20.0,9.0,91.511000,108.371000,0.844423,308.000000,16.717647,86.211765,0.980392,Finished,61,308.000000,0.844423,1
9,2022,Lando Norris,12,5,10.0,10.0,10.0,90.473000,108.175000,0.836358,298.000000,16.717647,86.211765,0.980392,Finished,130,301.000000,0.901863,1


## Scale values for temperature, humidity and fastest lap

Most of the time, scaling gives an improvement on model's accuracy. We imported a Standard Scaler and scaled down the features that have the most different ranges.

In [7]:
# Import Standard Scaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Create Scaler instances
scaler = StandardScaler()
fastest_lap_scaler = MinMaxScaler()

# Scaling Features - These will not be in X_2025 DataFrame so they are scaled separetly
features_to_scale = ['Temperature', 'Humidity', 'NumberOfRaces', 'MeanSpeedTrap']
final_sessions_encoded[features_to_scale] = scaler.fit_transform(final_sessions_encoded[features_to_scale])

# Scaling Lap Times
final_sessions_encoded['FastestLap'] = fastest_lap_scaler.fit_transform(final_sessions_encoded[['FastestLap']])

# Check if the scaling worked
print(final_sessions_encoded[['Temperature', 'Humidity', 'FastestLap','NumberOfRaces', 'MeanSpeedTrap']].head())


   Temperature  Humidity  FastestLap  NumberOfRaces  MeanSpeedTrap
0     -1.22512   1.45058    0.385728       0.314419       0.590784
1     -1.22512   1.45058    0.427347       1.042307       0.725158
2     -1.22512   1.45058    0.371200      -0.330281      -0.484211
3     -1.22512   1.45058    0.442459      -0.236696       0.456410
4     -1.22512   1.45058    0.407036       1.842983      -1.021709


## Creating and training a model

As we mentioned, for this project we decided to use Gradient Boosting, more precisely XGBoost. Here you can see the basic steps in creating and fitting a model. The explanation of features will be available in provided documentation. Please be aware that MAE will vary since not every race happened in every year from selected period. Some races only happened once in three previous years, so the training set will be smaller, causing MAE to increase.

In [8]:
# List of all used features - they are explained in the provided documentation
features = ['Abbreviation', 'YearWeight','TeamName', 'GridPosition', 'QualifyingTime', 'Temperature', 'Humidity', 'Rain', 'NumberOfRaces','MeanSpeedTrap', 'MeanTimeRatio']

# We are predicting final race position, here we could encode Status as well?
target = 'Position'

# Elementary data sets for machine learning model fitting
X = final_sessions_encoded[features]
y = final_sessions_encoded[target]

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 0)

# Training a model
model = GradientBoostingRegressor(n_estimators = 100, learning_rate = 0.1, random_state = 1)
model.fit(X_train, y_train)

# Prediction
y_pred = model.predict(X_test)
print("Mean Absolute Error for positions: ", mean_absolute_error(y_test,y_pred))

Mean Absolute Error for positions:  2.2701187881807177



<p style="text-align: left; font-size: 16px; color: #555; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;">
    With all this data available, we can also predict the fastest lap times of each driver. Let's create another model for that matter!
</p>


In [10]:
from sklearn.ensemble import RandomForestRegressor

# New target is the fastest lap time
target_lap = 'FastestLap'

# Elementary data sets for machine learning model fitting
X2 = final_sessions_encoded[features]
y2 = final_sessions_encoded[target_lap]

# Split the data into training and testing sets
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=0)

# Initialize the RandomForestRegressor model
model2 = RandomForestRegressor(random_state=1)

# Fit the model with the sample weights
model2.fit(X2_train, y2_train)

# Prediction on the test set
y2_pred = model2.predict(X2_test)

# Calculate error metrics
mae = mean_absolute_error(y2_test, y2_pred)
print(f"Mean Absolute Error for lap times: {mae}")

# Optionally: Display feature importances
importances = model2.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X2.columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print("Feature Importances:\n", feature_importance_df)

Mean Absolute Error for lap times: 0.035411965884185326
Feature Importances:
            Feature  Importance
6         Humidity    0.190933
7             Rain    0.170359
4   QualifyingTime    0.164916
10   MeanTimeRatio    0.164100
5      Temperature    0.131292
1       YearWeight    0.067631
9    MeanSpeedTrap    0.049392
0     Abbreviation    0.023354
8    NumberOfRaces    0.013267
2         TeamName    0.013075
3     GridPosition    0.011681


## Prediction Sample

So far, this prediction sample is created this way, we might try to automate the process. From the index 0 in each feature's list, we write data for the fastest driver of the qualifying session, then move on to second, and so on. Finally, predictions are available by the end of this cell.

In [12]:
import numpy as np

# Creating a sample dataset for 2025 predictions
data_2025 = {
    "Abbreviation": [22, 12, 15, 10, 17, 26, 28, 6, 0, 25],  # Already encoded driver names
    "YearWeight" : [5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
    "TeamName": [8, 5, 5, 2, 6, 6, 7, 2, 9, 3],  # Already encoded team names
    "GridPosition": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  # Qualifying grid positions
    "QualifyingTime": [86.983, 86.995, 87.027, 87.299, 87.318, 87.555, 87.569, 87.610, 87.615, 87.867],  # Lap times
    "Temperature": [19] * 10,  # Constant for the race
    "Humidity": [55] * 10,  # Constant for the race
    "Rain": [0.5] * 10,  # No rain
    "NumberOfRaces": [211, 130, 48, 149, 130, 2, 2, 358, 106, 5],  # Experience (Hadjar = 0)
    "MeanSpeedTrap": [324.000000, 318.333333, 321.000000, 320.666667, 319.0000000, 319.000000, 320.666667, 318.666667, 325.0000000, 322.0000000],  # Top speed
    "MeanTimeRatio": [0.947336, 0.951439, 0.947429, 0.943792, 0.948008, 0.948008, 0.942743, 0.952311, 0.951111, 0.936225]  # Lap consistency 
}

# Convert to DataFrame
X_2025 = pd.DataFrame(data_2025)

# Predict race positions and fastest laps using trained model
y_pred = model.predict(X_2025)
y2_pred = model2.predict(X_2025)

# OPTIONAL: Clip predictions to ensure valid positions (1–20 for example)
y_pred = np.clip(y_pred, 1, 20)

# Assign predictions back to DataFrame
X_2025["PredictedRacePosition"] = y_pred
X_2025["PredictedFastestLap"] = y2_pred

# Sort by predicted race position (ensuring ranking is unique)
X_2025 = X_2025.sort_values(by="PredictedRacePosition").reset_index(drop=True)

# Assign unique integer positions (1st to Nth) based on ranking
X_2025["FinalPredictedPosition"] = range(1, len(X_2025) + 1)

# Decode abbreviation from encoded integer back to original string
reverse_driver_encoding = {v: k for k, v in driver_encoding.items()}
X_2025['Abbreviation'] = X_2025['Abbreviation'].map(reverse_driver_encoding)

# Revert laps back to regular values 
X_2025['PredictedFastestLap'] = fastest_lap_scaler.inverse_transform(X_2025[['PredictedFastestLap']])

# Function to convert seconds to Min:Sec:Milliseconds
def convert_to_time_format(time_seconds):
    minutes = int(time_seconds // 60)
    seconds = int(time_seconds % 60)
    milliseconds = int((time_seconds - (minutes*60 + seconds))*1000)
    return f"{minutes}:{seconds:02d}:{milliseconds:03d}"

# Apply conversion to the PredictedFastestLap column
X_2025['FormattedFastestLap'] = X_2025['PredictedFastestLap'].apply(convert_to_time_format)

# Display the final sorted predictions
print(X_2025[['Abbreviation', 'PredictedRacePosition', 'FinalPredictedPosition', 'PredictedFastestLap', 'FormattedFastestLap']])



  Abbreviation  PredictedRacePosition  FinalPredictedPosition  \
0          NOR               1.815957                       1   
1          VER               1.830350                       2   
2          PIA               4.120683                       3   
3          LEC               4.429230                       4   
4          HAM               4.882088                       5   
5          HAD               9.100923                       6   
6          RUS               9.320713                       7   
7          ANT               9.356552                       8   
8          BEA               9.459625                       9   
9          ALB              11.574136                      10   

   PredictedFastestLap FormattedFastestLap  
0            99.942039            1:39:942  
1           100.038479            1:40:038  
2            99.887979            1:39:887  
3           100.009193            1:40:009  
4           100.109546            1:40:109  
5           10